### Logistic Regression for Classification

### Import Libraries

In [1]:
import numpy as np
import pandas as pd

### Load Dataset

In [2]:
"""Load data here"""
data = pd.read_csv('data.csv')

# Show the first 5 rows
print(data.head())

   Temperature   Humidity  Wind_Speed  Cloud_Cover     Pressure     Rain
0    23.720338  89.592641    7.335604    50.501694  1032.378759     rain
1    27.879734  46.489704    5.952484     4.990053   992.614190  no rain
2    25.069084  83.072843    1.371992    14.855784  1007.231620  no rain
3    23.622080  74.367758    7.050551    67.255282   982.632013     rain
4    20.591370  96.858822    4.643921    47.676444   980.825142  no rain


### Check for Missing Values

In [3]:
"""Check for Missing Data"""
# Step 3: Check for missing data
print("\n Missing values per column:")
print(data.isnull().sum())



 Missing values per column:
Temperature    0
Humidity       0
Wind_Speed     0
Cloud_Cover    0
Pressure       0
Rain           0
dtype: int64


### Perform Minmax Scaling

In [4]:
def minmax_scaling (data, column):
    min = data[column].min()
    max = data[column].max()
    return (data[column] - min) / (max - min)

In [5]:
"""Perform Minmax Scaling on Appropraite Columns"""
columns_to_scale = ['Temperature', 'Humidity', 'Wind_Speed', 'Cloud_Cover', 'Pressure']

# Apply scaling to each column
for col in columns_to_scale:
    data[col] = minmax_scaling(data, col)

# Preview the scaled dataset
print("\n Scaled Data:")
print(data.head())




 Scaled Data:
   Temperature  Humidity  Wind_Speed  Cloud_Cover  Pressure     Rain
0     0.548885  0.851343    0.366485     0.504954  0.748370     rain
1     0.715305  0.235520    0.297292     0.049759  0.180070  no rain
2     0.602850  0.758193    0.068145     0.148433  0.388977  no rain
3     0.544954  0.633821    0.352225     0.672518  0.037409     rain
4     0.423693  0.955157    0.231829     0.476696  0.011586  no rain


### Perform Encoding

In [6]:
"""Encode target column which is a categorical attribute (Hint: data[col].map)"""
# Clean and convert (Targetted column)'Rain' column to numeric
data['Rain'] = data['Rain'].str.strip().str.lower()  # Clean strings
data['Rain'] = data['Rain'].map({'rain': 1, 'no rain': 0})
print(data.head())

   Temperature  Humidity  Wind_Speed  Cloud_Cover  Pressure  Rain
0     0.548885  0.851343    0.366485     0.504954  0.748370     1
1     0.715305  0.235520    0.297292     0.049759  0.180070     0
2     0.602850  0.758193    0.068145     0.148433  0.388977     0
3     0.544954  0.633821    0.352225     0.672518  0.037409     1
4     0.423693  0.955157    0.231829     0.476696  0.011586     0


### Divide Data into Training and Testing

In [7]:
def train_test_split (data, ratio):
    indices = np.random.permutation(data.shape[0])
    test_set_size = int(data.shape[0] * ratio)
    test_indices = indices[:test_set_size]
    train_indices = indices[test_set_size:]

    return data.iloc[train_indices], data.iloc[test_indices]

In [8]:
"""Understand the above function and divide data into X_train, X_test, y_train, y_test"""
# Separate features and target
X = data.drop('Rain', axis=1)
y = data['Rain']

# Combine them for splitting (you can join and split, or split indices separately)
combined = pd.concat([X, y], axis=1)

# Use your function to split
train_data, test_data = train_test_split(combined, ratio=0.2)

# Separate X and y for training and testing
X_train = train_data.drop('Rain', axis=1)
y_train = train_data['Rain']
X_test = test_data.drop('Rain', axis=1)
y_test = test_data['Rain']

# Check shapes
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (2000, 5)
y_train shape: (2000,)
X_test shape: (500, 5)
y_test shape: (500,)


### Compute the Sigmoid Function

In [9]:
"""This function is used by consequent functions"""

def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar or numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z
    """
    g = 1 / (1 + np.exp(-z))
    return g

In [10]:
"""Test above function"""

value = 0
print (f"sigmoid({value}) = {sigmoid(value)}")

sigmoid(0) = 0.5


### Compute the Cost Function

In [11]:
"""This function is used by consequent functions"""

def compute_cost(X, y, theta):
    """
    Computes the logistic regression cost function

    Args:
      X : ndarray of shape (m, n), training data
      y : ndarray of shape (m,), target values (0 or 1)
      theta : ndarray of shape (n,), parameters

    Returns:
      total_cost : scalar, the logistic regression cost
    """
    m = y.shape[0]  # number of training examples

    h = sigmoid(np.dot(X, theta))  # predictions, shape (m,)

    # To avoid log(0), clip values between a small epsilon and 1-epsilon
    epsilon = 1e-15
    h = np.clip(h, epsilon, 1 - epsilon)

    cost = -(1/m) * (np.dot(y, np.log(h)) + np.dot((1 - y), np.log(1 - h)))

    return cost

In [12]:
"""Test above function"""
m, n = X_train.shape
initial_w = np.zeros(n)
cost = compute_cost(X_train, y_train, initial_w)
print('Cost at initial w: {:.3f}'.format(cost))

Cost at initial w: 0.693


### Compute Gradient of the Cost Function

In [13]:
"""This function is used by consequent function"""

def compute_gradient(X, y, w):
    """
    Computes the gradient for logistic regression 

    Args:
      X : ndarray of shape (m, n), data, m examples by n features
      y : ndarray of shape (m,), target values (0 or 1)
      w : ndarray of shape (n,), model parameters

    Returns:
      dj_dw : ndarray of shape (n,), the gradient of the cost w.r.t. parameters w
    """
    m = y.shape[0]  # number of training examples

    h = sigmoid(np.dot(X, w))  # predictions vector (m,)

    error = h - y  # difference between predictions and true labels (m,)

    dj_dw = (1 / m) * np.dot(X.T, error)  # gradient vector (n,)

    return dj_dw

In [14]:
"""Test above function"""
#Compute and display gradient with w and b initialized to zeros
# Suppose X_train shape is (m, n)
m, n = X_train.shape

# Add bias term (intercept) as a column of ones
X_train_bias = np.hstack((np.ones((m, 1)), X_train))  # shape (m, n+1)

# Initialize weights including bias term
initial_w = np.zeros(n + 1)

# Compute gradient
dj_dw = compute_gradient(X_train_bias, y_train, initial_w)

print(f'dj_dw at initial w: {dj_dw.tolist()}')

dj_dw at initial w: [0.379, 0.2139842990303756, 0.1500659903528615, 0.18687497410505072, 0.15560818845021596, 0.18585659942461802]


### Calcualte Weights Using Gradient Descent

In [20]:
def gradient_descent(X, y, w_in, cost_function, gradient_function, alpha, num_iters):
    """
    Performs batch gradient descent to learn theta. Updates theta by taking 
    num_iters gradient steps with learning rate alpha.
    
    Args:
      X : ndarray of shape (m, n), training data
      y : ndarray of shape (m,), target values
      w_in : ndarray of shape (n,), initial parameter values
      cost_function : function to compute cost
      gradient_function : function to compute gradient
      alpha : float, learning rate
      num_iters : int, number of iterations
      
    Returns:
      w : ndarray of shape (n,), updated parameters after gradient descent
    """
    w = w_in.copy()  # make a copy to avoid modifying original
    
    for i in range(num_iters):
        # Compute the gradient
        grad = gradient_function(X, y, w)
        
        # Update the parameters
        w -= alpha * grad
        
        # Optional: print cost every, say, 100 iterations for monitoring
        if i % 100 == 0 or i == num_iters - 1:
            cost = cost_function(X, y, w)
            print(f"Iteration {i}: Cost {cost}")
    
    return w

In [21]:
"""Call above function on your data with appropraite parameters and fetch the optimal weights."""
m, n = X_train.shape
X_train_bias = np.hstack((np.ones((m, 1)), X_train))  # add bias column
initial_w = np.zeros(n + 1)  # including bias weight
alpha = 0.01       # learning rate
num_iters = 1000   # number of gradient descent steps
optimal_w = gradient_descent(X_train_bias, y_train, initial_w, compute_cost, compute_gradient, alpha, num_iters)
print("Optimal weights:", optimal_w)

Iteration 0: Cost 0.6901752495495662
Iteration 100: Cost 0.5110277540264746
Iteration 200: Cost 0.4455496818611689
Iteration 300: Cost 0.4170230724045591
Iteration 400: Cost 0.40246406943067087
Iteration 500: Cost 0.3939083358653019
Iteration 600: Cost 0.3881885803234998
Iteration 700: Cost 0.3839073036541293
Iteration 800: Cost 0.3803991741939883
Iteration 900: Cost 0.3773292102265415
Iteration 999: Cost 0.37454868914707723
Optimal weights: [-0.92132719 -0.67258484 -0.07509013 -0.4094255  -0.11624506 -0.41273324]


### Calculate Predictions on Test Test

In [187]:
def predict(X, w):
    """
    Predict whether the label is 0 or 1 using learned logistic regression parameters w.
    
    Args:
      X : ndarray of shape (m, n), test data (including bias term if used)
      w : ndarray of shape (n,), model parameters
    
    Returns:
      p : ndarray of shape (m,), predictions (0 or 1)
    """
    probabilities = sigmoid(np.dot(X, w))
    p = (probabilities >= 0.5).astype(int)  # threshold at 0.5
    return p

### Calculate Accuracy of Model

In [188]:
"""Build a logic to estimate the model's accuracy"""
def accuracy(y_true, y_pred):
    """
    Compute the accuracy of predictions.

    Args:
      y_true : ndarray of shape (m,), true labels (0 or 1)
      y_pred : ndarray of shape (m,), predicted labels (0 or 1)

    Returns:
      acc : float, accuracy as a percentage (0-100)
    """
    correct = (y_true == y_pred).sum()
    total = y_true.shape[0]
    acc = (correct / total) * 100
    return acc
# Assuming you have test data X_test and true labels y_test
# Don't forget to add bias term to X_test if needed
m_test = X_test.shape[0]
X_test_bias = np.hstack((np.ones((m_test, 1)), X_test))

# Make predictions
y_pred = predict(X_test_bias, optimal_w)

# Calculate accuracy
acc = accuracy(y_test, y_pred)
print(f"Model accuracy: {acc:.2f}%")


Model accuracy: 88.00%
